# Exercise 4: Communication Generation & Action Planning

In the previous exercises, we've:
1. Collected comprehensive delivery data from BigQuery
2. Assessed delivery risks using external models
3. Analyzed products for special handling needs

Now we'll create **actionable communications** that GOAs can use immediately.

## The Journey So Far

```
1. Data Collection (Exercise 1) ✅ → collected_order_data.json
   ↓
2. Risk Assessment (Exercise 2) ✅ → risk_assessment_output.json
   ↓
3. Product Intelligence (Exercise 3) ✅ → product_intelligence_output.json
   ↓
4. Communication Generation (THIS EXERCISE) → communication_output.json
```

## Important: Order Selection
This exercise uses data from the previous exercises. Make sure you've run Exercises 1, 2, and 3 with your selected order first!

## Environment Setup

We'll configure the environment and import necessary libraries:

In [ ]:
import os
import warnings
import logging
import json
from typing import Dict, Any, List, Optional
from datetime import datetime, timedelta

# Suppress warnings
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.ERROR)

# Configure environment
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"
os.environ["GOOGLE_CLOUD_PROJECT"] = "traversaal-research"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

from google.adk.agents import Agent, SequentialAgent, ParallelAgent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

print("✅ Environment configured for communication generation")

## Understanding Communication Requirements

Home Depot has strict policies for customer and carrier communications. Let's load and understand these policies:

In [ ]:
# Load and display key policy points
try:
    with open('customer_communication_policy.md', 'r') as f:
        policy_content = f.read()
        
    print("📋 Key Policy Requirements:")
    print("\n1. REQUIRED ELEMENTS:")
    print("   - Order number reference")
    print("   - Delivery date")
    print("   - Clear next steps")
    print("   - Support contact info")
    print("\n2. PRO CUSTOMER BENEFITS:")
    print("   - Dedicated PRO desk contact")
    print("   - Extended support hours (5 AM - 9 PM)")
    print("   - Priority rescheduling")
    print("\n3. HIGH-RISK DELIVERIES:")
    print("   - Must state challenges explicitly")
    print("   - Require confirmation within 24 hours")
    print("   - Offer alternatives upfront")
    
except FileNotFoundError:
    print("⚠️ Policy file not found. Using default policy guidelines.")

## Communication Templates

Based on the policy, we have pre-approved templates for different scenarios:

In [ ]:
# Communication templates from policy
TEMPLATES = {
    "standard_confirmation": 
        "Hello {customer_name}, your Home Depot delivery #{order_number} is scheduled for {date}. "
        "{instructions}. Reply YES to confirm or CALL 1-800-HOME-DEPOT for changes.",
    
    "access_confirmation": 
        "Home Depot delivery #{order_number} scheduled for {date} requires special access to {location}. "
        "Please confirm: {access_need}. Reply with instructions or call {phone}.",
    
    "weather_contingency": 
        "Your Home Depot delivery #{order_number} may be affected by {weather} on {date}. "
        "Your {product_type} requires dry conditions. Reply 1 for next clear day, 2 to proceed, or 3 to call us.",
    
    "pro_notification": 
        "PRO DELIVERY: Order #{order_number} scheduled {date}. {details}. "
        "Direct line: 1-800-PRODEPOT (5AM-9PM). Reply CONFIRM or call for priority changes.",
    
    "carrier_alert": 
        "DELIVERY ALERT #{order_number}: {date} to {address}. Weight: {weight}lbs, {equipment_req}. "
        "Customer notes: {notes}. Risk: {risk_level}. Issues? Call dispatch: {dispatch_phone}"
}

# Support numbers
SUPPORT_NUMBERS = {
    "standard": "1-800-HOME-DEPOT",
    "pro": "1-800-PRODEPOT",
    "dispatch": "1-800-DISPATCH",
    "priority": "1-800-PRIORITY"
}

print("📱 Available message templates:")
for template_name in TEMPLATES:
    print(f"   - {template_name}")

## Loading Previous Pipeline Outputs

In a production system, this data would flow automatically. For the exercise, we'll load the outputs from previous pipelines:

In [ ]:
# Load data from previous exercises
def load_pipeline_data():
    """Load outputs from previous pipeline stages"""
    data = {}
    
    # Try to load actual outputs
    try:
        with open('../exercise_1_data_collection/collected_order_data.json', 'r') as f:
            data['order_data'] = json.load(f)
            print("✅ Loaded order data from Exercise 1")
    except:
        # Fallback mock data
        data['order_data'] = {
            "order": {
                "CUSTOMER_ORDER_NUMBER": "CG92094171",
                "SCHEDULED_DELIVERY_DATE": "2025-06-21",
                "WEIGHT": 1598,
                "VEHICLE_TYPE": "FLAT"
            },
            "customer": {
                "CUSTOMER_NAME": "CUST_01518",
                "PRO_XTRA_MEMBER": True,
                "DESTINATION_ADDRESS": "668 FOREST AVE ELGIN, IL 60120",
                "CUSTOMER_NOTES": "call b/4 delivery delivery from the back of the building"
            },
            "environmental": {
                "WTHR_CATEGORY": "Clear",
                "PRECIPITATION": 0.09,
                "STRT_VW_IMG_DSCRPTN": "* The driveway is partially obscured by trees"
            }
        }
        print("⚠️ Using mock order data")
    
    try:
        with open('../exercise_2_risk_assessment/risk_assessment_output.json', 'r') as f:
            data['risk_assessment'] = json.load(f)
            print("✅ Loaded risk assessment from Exercise 2")
    except:
        data['risk_assessment'] = {
            "risk_assessment": {
                "risk_level": "MEDIUM",
                "overall_risk_score": 7,
                "risk_factors": ["Heavy load", "Special instructions"],
                "recommendations": [
                    {
                        "action": "Schedule early morning delivery",
                        "priority": "HIGH",
                        "reason": "Avoid residential traffic"
                    }
                ]
            }
        }
        print("⚠️ Using mock risk assessment")
    
    try:
        with open('../exercise_3_product_intelligence/product_intelligence_output.json', 'r') as f:
            data['product_intelligence'] = json.load(f)
            print("✅ Loaded product intelligence from Exercise 3")
    except:
        data['product_intelligence'] = {
            "priority_scoring": {
                "priority_score": 65,
                "priority_level": "MEDIUM"
            },
            "product_analysis": {
                "weather_sensitive": True,
                "weather_concerns": ["lumber - protect from rain"],
                "handling_requirements": ["Heavy lifting equipment"]
            },
            "vehicle_compatibility": {
                "vehicle_appropriate": True,
                "issues": [],
                "recommendations": []
            }
        }
        print("⚠️ Using mock product intelligence")
    
    return data

# Load all data
pipeline_data = load_pipeline_data()

print("\n📊 Data Summary:")
print(f"Order: {pipeline_data['order_data']['order']['CUSTOMER_ORDER_NUMBER']}")
print(f"Risk Level: {pipeline_data['risk_assessment']['risk_assessment']['risk_level']}")
print(f"Customer Type: {'PRO' if pipeline_data['order_data']['customer']['PRO_XTRA_MEMBER'] else 'Standard'}")
print(f"Priority Score: {pipeline_data['product_intelligence'].get('priority_scoring', {}).get('priority_score', 'N/A')}")

## Policy Compliance Tools

First, let's create tools to ensure all communications comply with company policy:

In [ ]:
def check_policy_compliance(message: str, message_type: str) -> Dict[str, Any]:
    """Check if message complies with company policy"""
    compliance = {
        "compliant": True,
        "issues": [],
        "suggestions": []
    }
    
    # Check required elements
    if "#" not in message:
        compliance["compliant"] = False
        compliance["issues"].append("Missing order number reference")
        
    # Check prohibited content
    prohibited_terms = ["driver name", "guarantee", "promise", "definitely", "100%", "competitor"]
    for term in prohibited_terms:
        if term.lower() in message.lower():
            compliance["compliant"] = False
            compliance["issues"].append(f"Contains prohibited term: {term}")
            
    # Check message length for SMS
    if len(message) > 160:
        compliance["suggestions"].append(f"Consider shortening message for SMS (currently {len(message)} chars)")
        
    # Check for support contact
    if not any(num in message for num in SUPPORT_NUMBERS.values()):
        compliance["suggestions"].append("Consider adding support contact information")
        
    return compliance

# Test the compliance checker
test_message = "Your delivery is tomorrow. Call if needed."
compliance_result = check_policy_compliance(test_message, "customer")
print("Test message compliance:")
print(json.dumps(compliance_result, indent=2))

## Message Formatting Tools

Now let's create tools to format messages using our templates:

In [ ]:
def format_customer_message(
    template_key: str,
    order_data: Dict[str, Any],
    risk_data: Dict[str, Any],
    product_data: Optional[Dict[str, Any]] = None,
    **kwargs
) -> str:
    """Format customer message using templates"""
    
    template = TEMPLATES.get(template_key, TEMPLATES["standard_confirmation"])
    
    # Extract common fields
    order = order_data.get("order", {})
    customer = order_data.get("customer", {})
    
    # Build format dictionary
    format_dict = {
        "customer_name": customer.get("CUSTOMER_NAME", "Valued Customer"),
        "order_number": order.get("CUSTOMER_ORDER_NUMBER", ""),
        "date": order.get("SCHEDULED_DELIVERY_DATE", ""),
        "phone": SUPPORT_NUMBERS["pro"] if customer.get("PRO_XTRA_MEMBER") else SUPPORT_NUMBERS["standard"],
        **kwargs  # Allow additional custom fields
    }
    
    # Format message
    try:
        message = template.format(**format_dict)
    except KeyError as e:
        # Return safe fallback if template formatting fails
        message = f"Home Depot delivery #{format_dict['order_number']} update. Please call {format_dict['phone']} for details."
        
    return message

# Test message formatting
test_msg = format_customer_message(
    "standard_confirmation",
    pipeline_data['order_data'],
    pipeline_data['risk_assessment'],
    instructions="Please ensure someone is available to receive the delivery"
)
print("Sample formatted message:")
print(test_msg)
print(f"\nMessage length: {len(test_msg)} characters")

## Alternative Solution Generation

For high-risk deliveries, we need to suggest alternatives:

In [ ]:
def generate_alternative_dates(original_date: str, weather_forecast: Optional[Dict[str, Any]] = None) -> List[str]:
    """Generate alternative delivery dates based on constraints"""
    try:
        # Parse original date
        date_obj = datetime.strptime(original_date, "%Y-%m-%d")
    except:
        # Fallback if date parsing fails
        date_obj = datetime.now()
        
    alternatives = []
    
    # Generate 3 alternative dates within 7 days
    for i in range(1, 8):
        alt_date = date_obj + timedelta(days=i)
        # Skip weekends for standard deliveries
        if alt_date.weekday() < 5:  # Monday = 0, Friday = 4
            alternatives.append(alt_date.strftime("%Y-%m-%d"))
            if len(alternatives) >= 3:
                break
                
    return alternatives


def suggest_alternatives(
    order_data: Dict[str, Any],
    risk_data: Dict[str, Any],
    product_data: Optional[Dict[str, Any]] = None
) -> List[Dict[str, Any]]:
    """Suggest alternative delivery solutions"""
    
    alternatives = []
    risk_assessment = risk_data.get("risk_assessment", {})
    order = order_data.get("order", {})
    
    # Date change alternatives
    if risk_assessment.get("risk_level") in ["HIGH", "MEDIUM"]:
        alt_dates = generate_alternative_dates(order.get("SCHEDULED_DELIVERY_DATE", ""))
        alternatives.append({
            "type": "reschedule",
            "description": "Reschedule to lower-risk date",
            "options": alt_dates,
            "approval_required": False
        })
        
    # Split delivery for large orders
    if order.get("WEIGHT", 0) > 3000 or order.get("QUANTITY", 0) > 100:
        alternatives.append({
            "type": "split_delivery",
            "description": "Split into multiple smaller deliveries",
            "options": ["2 deliveries", "3 deliveries"],
            "approval_required": False
        })
        
    # Hold for pickup
    if risk_assessment.get("risk_level") == "HIGH":
        alternatives.append({
            "type": "hold_pickup",
            "description": "Hold at store for customer pickup",
            "options": ["Will Call desk", "PRO desk pickup"],
            "approval_required": False
        })
        
    return alternatives

# Test alternative generation
alternatives = suggest_alternatives(
    pipeline_data['order_data'],
    pipeline_data['risk_assessment']
)
print("Suggested alternatives:")
for alt in alternatives:
    print(f"\n- {alt['description']}")
    print(f"  Options: {alt['options']}")
    print(f"  Approval needed: {alt['approval_required']}")

## Building Communication Generation Agents

Now let's create specialized agents for different communication types:

In [ ]:
GEMINI_MODEL = "gemini-2.0-flash"

# Customer message generator agent
customer_message_agent = Agent(
    model=GEMINI_MODEL,
    name="customer_message_agent",
    description="Generates customer communications",
    instruction="""\
You receive comprehensive delivery data including order, risk, and product information.

Based on the risk level and customer type, generate appropriate customer messages:

1. For HIGH risk deliveries:
   - Use weather_contingency or access_confirmation templates
   - Include specific challenges and alternatives
   - Add priority support number

2. For PRO customers:
   - Use pro_notification template
   - Include extended support hours
   - Offer direct coordination options

3. For standard deliveries:
   - Use standard_confirmation template
   - Include special instructions if any
   - Keep message concise

Use the format_customer_message tool with appropriate template and data.

Generate 1-2 messages maximum, focusing on the most critical communication needs.

Return messages in JSON format:
{
    "customer_messages": [
        {
            "type": "primary|followup",
            "template_used": "<template_key>",
            "message": "<formatted message>",
            "send_timing": "immediate|1_day_before|morning_of"
        }
    ]
}
""",
    tools=[format_customer_message],
    output_key="customer_messages"
)

# Policy compliance checker agent
policy_checker_agent = Agent(
    model=GEMINI_MODEL,
    name="policy_checker_agent",
    description="Checks messages for policy compliance",
    instruction="""\
You receive draft messages and check them against company policy.

Use the check_policy_compliance tool on each message.

If non-compliant, suggest corrections that maintain the message intent while following policy.

Return compliance status and any revised messages.
""",
    tools=[check_policy_compliance],
    output_key="policy_compliance"
)

# Alternative solution agent
alternative_solution_agent = Agent(
    model=GEMINI_MODEL,
    name="alternative_solution_agent",
    description="Suggests alternative delivery solutions",
    instruction="""\
You receive delivery data and risk assessments.

Use the suggest_alternatives tool to generate delivery alternatives.

Prioritize alternatives by:
1. Ease of implementation (no approval needed first)
2. Risk reduction potential
3. Customer convenience

For each alternative, provide clear benefits and implementation steps.

Return in JSON format:
{
    "alternative_solutions": <output from suggest_alternatives tool>,
    "recommended_action": {
        "primary_recommendation": "<best alternative>",
        "reason": "<why this is best>",
        "implementation": "<how to implement>"
    }
}
""",
    tools=[suggest_alternatives, generate_alternative_dates],
    output_key="alternative_solutions"
)

print("✅ Communication agents created")
print("\nAgents:")
print("1. Customer Message Agent - Generates customer notifications")
print("2. Policy Checker Agent - Ensures compliance")
print("3. Alternative Solution Agent - Suggests risk mitigation options")

## Carrier Communication Tools

Carriers need specific, actionable instructions:

In [ ]:
def generate_carrier_instructions(
    order_data: Dict[str, Any],
    risk_data: Dict[str, Any],
    product_data: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """Generate specific instructions for carrier"""
    
    order = order_data.get("order", {})
    customer = order_data.get("customer", {})
    environmental = order_data.get("environmental", {})
    risk_assessment = risk_data.get("risk_assessment", {})
    
    instructions = {
        "priority": "STANDARD",
        "equipment_needed": [],
        "access_notes": [],
        "risk_mitigation": [],
        "contact_protocol": []
    }
    
    # Set priority based on risk and customer type
    if risk_assessment.get("risk_level") == "HIGH":
        instructions["priority"] = "HIGH"
    elif customer.get("PRO_XTRA_MEMBER"):
        instructions["priority"] = "PRO"
        
    # Equipment requirements based on weight
    weight = order.get("WEIGHT", 0)
    if weight > 3000:
        instructions["equipment_needed"].append("Heavy lift equipment required")
    if weight > 1500:
        instructions["equipment_needed"].append("2-person team recommended")
        
    # Access notes from street view
    street_desc = environmental.get("STRT_VW_IMG_DSCRPTN", "")
    if "limited" in street_desc.lower():
        instructions["access_notes"].append("Limited access - plan route carefully")
    if "dead end" in street_desc.lower():
        instructions["access_notes"].append("Dead end street - ensure turnaround space")
        
    # Customer special instructions
    if customer.get("CUSTOMER_NOTES"):
        instructions["access_notes"].append(f"Customer request: {customer['CUSTOMER_NOTES']}")
        
    # Risk mitigation
    for risk in risk_assessment.get("risk_factors", []):
        if "weather" in risk.lower():
            instructions["risk_mitigation"].append("Monitor weather conditions")
        elif "heavy" in risk.lower():
            instructions["risk_mitigation"].append("Confirm equipment before departure")
            
    # Contact protocol
    if risk_assessment.get("risk_level") == "HIGH":
        instructions["contact_protocol"].append("Call customer 1 hour before arrival")
    if customer.get("CUSTOMER_NOTES") and "call" in customer.get("CUSTOMER_NOTES", "").lower():
        instructions["contact_protocol"].append("Customer requests call before delivery")
        
    return instructions

# Test carrier instruction generation
carrier_instructions = generate_carrier_instructions(
    pipeline_data['order_data'],
    pipeline_data['risk_assessment']
)
print("Generated carrier instructions:")
print(json.dumps(carrier_instructions, indent=2))

## Complete Communication Pipeline

Now let's orchestrate all agents into a complete pipeline:

In [ ]:
# Carrier instruction agent
carrier_instruction_agent = Agent(
    model=GEMINI_MODEL,
    name="carrier_instruction_agent",
    description="Generates carrier-specific delivery instructions",
    instruction="""\
You receive delivery data and risk assessments.

Use the generate_carrier_instructions tool to create detailed carrier guidance.

Focus on:
1. Equipment requirements based on weight/volume
2. Access challenges from street view data
3. Customer special requests
4. Risk mitigation steps

Also create a brief carrier message using the carrier_alert template via format_customer_message.

Return in JSON format:
{
    "carrier_communication": {
        "alert_message": "<formatted carrier alert>",
        "detailed_instructions": <output from generate_carrier_instructions tool>,
        "dispatch_priority": "STANDARD|HIGH|URGENT"
    }
}
""",
    tools=[generate_carrier_instructions, format_customer_message],
    output_key="carrier_communication"
)

# Final communication assembler agent
communication_assembler_agent = Agent(
    model=GEMINI_MODEL,
    name="communication_assembler_agent",
    description="Assembles all communications into final output",
    instruction="""\
You receive all generated communications and compile them into a structured output.

Review all inputs:
- {customer_messages}
- {carrier_communication}
- {alternative_solutions}
- {policy_compliance}

Create a final JSON structure that includes:

{
    "communications": {
        "order_number": "<from order data>",
        "risk_level": "<from risk assessment>",
        "customer_messages": <from customer_message_agent>,
        "carrier_instructions": <from carrier_instruction_agent>,
        "alternatives": <from alternative_solution_agent>,
        "compliance_check": <from policy_checker_agent if any issues>,
        "action_summary": {
            "immediate_actions": [<list actions to take now>],
            "scheduled_actions": [<list actions for later>],
            "contingency_plans": [<backup options if needed>]
        }
    }
}

Return ONLY the JSON structure.
""",
    tools=[],
    output_key="final_communications"
)

# Parallel generation of communications
communication_generators = ParallelAgent(
    name="communication_generators",
    sub_agents=[customer_message_agent, carrier_instruction_agent, alternative_solution_agent],
    description="Generate all communication types in parallel"
)

# Sequential pipeline
communication_pipeline = SequentialAgent(
    name="communication_pipeline",
    sub_agents=[communication_generators, policy_checker_agent, communication_assembler_agent],
    description="Complete communication generation pipeline"
)

print("✅ Communication pipeline assembled")
print("\nPipeline flow:")
print("1️⃣  Parallel: Customer + Carrier + Alternative messages")
print("2️⃣  Policy compliance check")
print("3️⃣  Final assembly → Structured output")

## Running the Communication Pipeline

Let's execute the complete communication generation:

In [ ]:
import sys
import io

async def run_communication_generation(
    order_data: Dict[str, Any],
    risk_assessment: Dict[str, Any],
    product_intelligence: Dict[str, Any]
):
    """Run communication generation pipeline"""
    
    # Setup
    session_service = InMemorySessionService()
    await session_service.create_session(
        app_name="communication_generation",
        user_id="user_1",
        session_id="comm_session_001"
    )
    
    # Prepare combined data
    combined_data = {
        "order_data": order_data,
        "risk_assessment": risk_assessment,
        "product_intelligence": product_intelligence
    }
    
    runner = Runner(
        agent=communication_pipeline,
        app_name="communication_generation",
        session_service=session_service
    )
    
    print("=" * 60)
    print("COMMUNICATION GENERATION PIPELINE")
    print("=" * 60)
    print(f"\nGenerating communications for order: {order_data.get('order', {}).get('CUSTOMER_ORDER_NUMBER', 'Unknown')}")
    print(f"Risk Level: {risk_assessment.get('risk_assessment', {}).get('risk_level', 'Unknown')}")
    print(f"Customer Type: {'PRO' if order_data.get('customer', {}).get('PRO_XTRA_MEMBER') else 'Standard'}")
    print("\nGenerating messages...\n")
    
    # Create message
    content = types.Content(
        role="user",
        parts=[types.Part(text=json.dumps(combined_data))]
    )
    
    # Run pipeline
    old_stderr = sys.stderr
    sys.stderr = io.StringIO()
    
    try:
        async for event in runner.run_async(
            user_id="user_1",
            session_id="comm_session_001",
            new_message=content
        ):
            sys.stderr = old_stderr
            
            if hasattr(event, "author") and event.author:
                if event.author in ["customer_message_agent", "carrier_instruction_agent", 
                                  "alternative_solution_agent", "policy_checker_agent"]:
                    print(f"[{event.author}] generating...")
                    
            if event.is_final_response() and event.author == "communication_assembler_agent":
                if event.content and event.content.parts:
                    print("\n" + "=" * 60)
                    print("COMMUNICATION OUTPUT")
                    print("=" * 60)
                    
                    # Parse and save JSON
                    try:
                        response_text = event.content.parts[0].text.strip()
                        if response_text.startswith("```json"):
                            response_text = response_text[7:]
                        if response_text.endswith("```"):
                            response_text = response_text[:-3]
                        
                        json_data = json.loads(response_text.strip())
                        
                        # Save to file
                        with open('communication_output.json', 'w') as f:
                            json.dump(json_data, f, indent=2)
                        
                        # Display key outputs
                        comms = json_data.get('communications', {})
                        
                        print("\n📱 CUSTOMER MESSAGE:")
                        for msg in comms.get('customer_messages', []):
                            print(f"\n[{msg.get('send_timing', 'immediate').upper()}]")
                            print(msg.get('message', ''))
                            
                        print("\n🚚 CARRIER INSTRUCTIONS:")
                        carrier = comms.get('carrier_instructions', {})
                        print(f"Priority: {carrier.get('dispatch_priority', 'STANDARD')}")
                        print(f"Alert: {carrier.get('alert_message', '')}")
                        
                        print("\n💡 RECOMMENDED ALTERNATIVES:")
                        alts = comms.get('alternatives', [])
                        for alt in alts[:3]:  # Show top 3
                            print(f"- {alt.get('solution', alt.get('description', ''))}")
                            
                        print("\n✅ ACTION SUMMARY:")
                        actions = comms.get('action_summary', {})
                        for action in actions.get('immediate_actions', [])[:2]:
                            print(f"• {action}")
                            
                        print("\n✅ Full output saved to communication_output.json")
                    except Exception as e:
                        print("Error parsing JSON:", e)
                        print(event.content.parts[0].text)
                break
                
            sys.stderr = io.StringIO()
    finally:
        sys.stderr = old_stderr

# Run the pipeline
await run_communication_generation(
    pipeline_data['order_data'],
    pipeline_data['risk_assessment'],
    pipeline_data['product_intelligence']
)

## Key Takeaways

This exercise demonstrated several important capabilities:

### 1. **Policy-Driven Communication**
- All messages automatically comply with company policies
- Templates ensure consistent messaging
- Compliance checking prevents policy violations

### 2. **Contextual Intelligence**
- PRO customers get different messaging than standard customers
- High-risk deliveries trigger proactive communications
- Weather-sensitive products get appropriate warnings

### 3. **Actionable Outputs**
- GOAs receive ready-to-send messages
- Carriers get specific, detailed instructions
- Alternative solutions are pre-approved and ready

### 4. **Efficiency Gains**
```python
# Before: 5-10 minutes per case
# - Read through order details
# - Check policy requirements
# - Draft appropriate messages
# - Think of alternatives

# After: < 10 seconds
# - Everything generated automatically
# - Policy compliance guaranteed
# - Best practices embedded
```

## Production Enhancements

In a production environment, you could enhance this system with:

1. **Message Delivery Integration**
   - Direct SMS/email sending
   - Customer preference management
   - Delivery confirmation tracking

2. **Feedback Loop**
   - Track message effectiveness
   - A/B test different templates
   - Learn from successful deliveries

3. **Multi-language Support**
   - Spanish language templates
   - Language detection from customer data
   - Cultural communication preferences

4. **Advanced Personalization**
   - Customer history integration
   - Previous delivery preferences
   - Seasonal considerations

## Next Steps

In the final exercise, we'll combine all components into a single, end-to-end delivery intelligence system that produces complete case cards for GOAs!

## Try It Yourself!

> **⚠️ Important**: Complete all 5 exercises before trying these modifications! The exercises build on each other's outputs. Changing code here will affect the data flow to subsequent exercises.

Practice modifying the communication generation system:

### Exercise 1: Add a New Message Template
Add a new template for "delivery_complete" in the TEMPLATES dictionary:
```python
"delivery_complete": 
    "Thank you {customer_name}! Your Home Depot order #{order_number} has been delivered. "
    "Questions? Call {phone}. Rate your experience at homedepot.com/feedback."
```

### Exercise 2: Adjust Risk-Based Messaging
Modify the customer_message_agent instruction to:
- Send TWO messages for HIGH risk deliveries (immediate + 1 day before)
- Add weather alerts for weather-sensitive products
- Include estimated delivery time window

### Exercise 3: Enhance Carrier Instructions
Add a new check in `generate_carrier_instructions`:
```python
# If delivery window is early morning (before 8 AM)
if order.get("WINDOW_START", "09:00") < "08:00":
    instructions["contact_protocol"].append("No doorbell before 8 AM - call instead")
```

### Challenge: Create a Spanish Language Option
1. Add Spanish translations to key templates
2. Check if customer prefers Spanish (you can add a mock field)
3. Generate bilingual messages when needed
4. Test with a PRO customer who prefers Spanish communication